In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from pandas_profiling import ProfileReport
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.options.display.max_colwidth = 1000
%matplotlib inline

In [38]:
# Functions
def get_contengcy_matrix(df1, df2):
    return [[(df1 == 'yes').sum(), (df2 == 'yes').sum()], [(df1 == 'no').sum(), (df2 == 'no').sum()]]

In [2]:
data_path = "../data_cleaning/clean_data/"
df_desc = pd.read_csv(data_path+"description.csv")
df_perfusion_v1 = pd.read_csv(data_path+"df_perfusion_model.csv", sep=",", header=0, index_col="patient id")
df_stroke_v1 =  pd.read_csv(data_path+"df_stroke_model.csv", sep=",", header=0, index_col="patient id")

In [3]:
df_desc.head()

,patient id,s0328,s0342,variable description,units
0,NaN,NaN,NaN,NaN,NaN
1,study,ge75,ge75,study number,NaN
2,completed visits status,completed,completed,completed study status,NaN
3,htn patient medical history,no,no,hypertension patient history,NaN
4,group 2,control,dm,group control diabetes,NaN


In [4]:
df_perfusion_v1.shape

(56, 42)

In [5]:
df_stroke_v1.head()

,stroke patient medical history,age,height,mass,bmi,gender,years,previous alcohol use,alcohol dose/week,neuropathy autonomic symptoms,dizziness autonomic symptoms,numbness autonomic symptoms,painful feet autonomic symptoms,syncope autonomic symptoms,oh autonomic symptoms,cancer# family history,heartdisease family history,htn family history,dm family history,strokefamily history,cancer patient medical history,atrial fibtrillation patient medical history,heart failure =chf /ifarction=mi patient medical history,hyperlipidemia patient medical history,diabetes,antiplatelets,anticoagulants,antihyperlipidemic,statins,estrogen,ace inhibitors,arbs,beta blockers,diuretics,ca ++ blockers,oral hypoglycemic
patient id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
s0011,no,70,1.6383,71.667594,26.701519,female,40.0,yes,0.0,no,no,no,no,no,no,0.0,1.0,0.0,0.0,0.0,no,no,no,no,no,no,no,no,no,no,no,yes,no,no,no,no
s0033,no,65,1.8200,92.986000,28.072093,male,10.0,yes,1.0,no,no,no,no,no,no,0.0,2.0,1.0,1.0,1.0,no,no,no,no,no,no,no,no,no,no,yes,no,no,no,no,no
s0044,no,74,1.7018,68.038855,23.493080,female,40.0,yes,2.0,no,yes,yes,no,no,yes,3.0,0.0,0.0,0.0,0.0,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no
s0067,no,70,1.6764,68.492448,24.371788,male,0.0,yes,7.0,no,no,no,no,no,no,1.0,0.0,0.0,0.0,1.0,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no
s0068,no,79,1.5748,64.863709,26.154774,female,0.0,no,0.0,no,no,no,no,no,no,3.0,0.0,0.0,0.0,0.0,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no,no


# Introduction 

This jupyter notebook aims to do a statistical analysis of datasets further used for stroke modeling. 
First let's use pandas_profling to have an overview.

We know that perfusion and stroke dataset have uncommon variables

In [6]:
set(df_perfusion_v1.columns).difference(df_stroke_v1.columns)

{'antiparkinsonian',
 'current tobacco use',
 'diabetes duration',
 'insulin(yes or no)',
 'pack years',
 'previous tobacco use'}

In [7]:
profile_perfusion = ProfileReport(df_perfusion_v1.reset_index(drop=True))
profile_perfusion

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
profile_stroke = ProfileReport(df_stroke_v1.reset_index(drop=True))
profile_stroke

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
#profile_perfusion.to_file(output_file='profile_perfusion_model.html')
#profile_stroke.to_file(output_file='profile_stroke_model.html')

The main difference between the two datasets is one contains patients 50/56 diabetes (perfusion dataset) and the other one amlost don't 3/112(stroke dataset). Therefore we want to compare some of the most interesting variables between the two and see how they differ from each other. 

We know that age is a major risk factor: 
Perfusion patients were between 50 and 80 and average 65 and stroke dataset's patients between 50 and 83 years old and are on average 66. There belongs to same age group.

# Obesity related features

Interestingly, there seems to be a difference of perfusion and stroke patients regarding bmi. Since wouldn't be surprinsing since we know obesity is a common comorbidity in T2DM.

In [26]:
print(stats.shapiro(df_perfusion_v1["bmi"].to_numpy()))
print(stats.shapiro(df_stroke_v1["bmi"].to_numpy()))

ShapiroResult(statistic=0.9242681860923767, pvalue=0.0017525386065244675)
ShapiroResult(statistic=0.9467765092849731, pvalue=0.0002188966900575906)


p.values < 0.05 so both variables don't follow a Normal Law. Lett's use a non parametric test to see if perfusion patients have a significantly higher bmi

In [16]:
stats.mannwhitneyu(df_perfusion_v1["bmi"].to_numpy(), df_stroke_v1["bmi"].to_numpy(), alternative="greater")

MannwhitneyuResult(statistic=2913.0, pvalue=0.4377252768321782)

pvalue < 0.05 rejects the null hypothesis, perfusion patients have a higher bmi than stroke patients 

Let's also compare hyperlipidemia prevelance between the two groups since it's related to obesity.

In [44]:
#Let's see for hyperlipidemia

oddsratio, pvalue = stats.fisher_exact(get_contengcy_matrix(df_stroke_v1["hyperlipidemia patient medical history"], df_perfusion_v1["hyperlipidemia patient medical history"] ))
print(f"hyperlipidemia\noddsratio: {oddsratio} \npvalue: {pvalue}")

oddsratio, pvalue = stats.fisher_exact(get_contengcy_matrix(df_stroke_v1["antihyperlipidemic"], df_perfusion_v1["antihyperlipidemic"] ))
print(f"\nantihyperlipidemia drug\noddsratio: {oddsratio} \npvalue: {pvalue}")

oddsratio, pvalue = stats.fisher_exact(get_contengcy_matrix(df_stroke_v1["statins"], df_perfusion_v1["statins"] ))
print(f"\nstatins\noddsratio: {oddsratio} \npvalue: {pvalue}")

hyperlipidemia
oddsratio: 0.5409836065573771 
pvalue: 0.07302206991271117

antihyperlipidemia drug
oddsratio: 0.15728155339805824 
pvalue: 1.88774820359914e-05

statins
oddsratio: 0.5025712949976625 
pvalue: 0.047782620400378714


H0: independance of distribution of feature X between the 2 datasets

Hyperlipidemia: pvalue > 0.05 so there's no difference between the two datasets regarding hyperlipidemia. 
However, people in the perfusion datasets are more prone to take antihyperlipidemia medication and statins (pvalue < 0.05) . 

# Lifestyle features - alcohol and smoking

In [39]:
print(stats.shapiro(df_perfusion_v1["alcohol dose/week"].to_numpy()))
print(stats.shapiro(df_stroke_v1["alcohol dose/week"].to_numpy()))

ShapiroResult(statistic=0.4526926279067993, pvalue=3.8712972714669003e-13)
ShapiroResult(statistic=0.4470900297164917, pvalue=1.1568691621176754e-18)


In [40]:
stats.mannwhitneyu(df_perfusion_v1["alcohol dose/week"].to_numpy(), df_stroke_v1["alcohol dose/week"].to_numpy(), alternative="two-sided")

MannwhitneyuResult(statistic=2913.0, pvalue=0.4377252768321782)

In [41]:
print(stats.shapiro(df_perfusion_v1["years"].to_numpy()))
print(stats.shapiro(df_stroke_v1["years"].to_numpy()))

ShapiroResult(statistic=0.7479636669158936, pvalue=1.889381628927822e-08)
ShapiroResult(statistic=0.8127068281173706, pvalue=1.2698096740759723e-10)


In [42]:
stats.mannwhitneyu(df_perfusion_v1["years"].to_numpy(), df_stroke_v1["years"].to_numpy(), alternative="two-sided")

MannwhitneyuResult(statistic=2765.0, pvalue=0.18760574988102496)

There is no significant variation of alcohol consumption nor smoking.

# Blood Pressure

ARBS: Angiotensin Receptor Blockers

ACE inhibitors: lower blood pressure by preventing the production of angiotensin II, a substance that narrows the blood vessels, while ARBs reduce the action of angiotensin II to prevent blood vessel constriction.

beta blockers: used to manage abnormal heart rhythms, and to protect the heart from a second heart attack (myocardial infarction) after a first heart attack (secondary prevention). They are also widely used to treat high blood pressure (hypertension), although they are no longer the first choice for initial treatment of most patients.

Diuretic:  the reduction in blood pressure is not due to decreased blood volume resulting from increased urine production, but occurs through other mechanisms and at lower doses than that required to produce diuresis

ca ++ blockers:   Calcium causes the heart and arteries to squeeze (contract) more strongly

In [76]:
htn_medication_perfusion = pd.DataFrame(df_perfusion_v1.groupby(["arbs", 'ace inhibitors', 'beta blockers', "diuretics", "ca ++ blockers"]).size())
htn_medication_perfusion[0]/len(df_perfusion_v1)


arbs  ace inhibitors  beta blockers  diuretics  ca ++ blockers
no    no              no             no         no                0.482143
                                                yes               0.035714
                                     yes        no                0.017857
                      yes            no         no                0.089286
                                     yes        no                0.017857
      yes             no             no         no                0.178571
                                     yes        yes               0.017857
                      yes            no         no                0.035714
                                                yes               0.017857
yes   no              no             no         no                0.035714
                                                yes               0.017857
                      yes            no         no                0.017857
                                     

In [74]:
htn_medication_stroke = pd.DataFrame(df_stroke_v1.groupby(["arbs", 'ace inhibitors', 'beta blockers', "diuretics", "ca ++ blockers"]).size())
htn_medication_stroke[0]/len(df_stroke_v1)

arbs  ace inhibitors  beta blockers  diuretics  ca ++ blockers
no    no              no             no         no                0.428571
                                                yes               0.008929
                                     yes        no                0.071429
                                                yes               0.035714
                      yes            no         no                0.080357
                                                yes               0.017857
                                     yes        no                0.008929
                                                yes               0.017857
      yes             no             no         no                0.107143
                                                yes               0.035714
                                     yes        no                0.017857
                                                yes               0.026786
                      yes            

Patients are taking multiple combination of medications.

In [78]:
print(htn_medication_stroke.shape)
print(htn_medication_perfusion.shape)

(20, 1)
(14, 1)


There are different combination of medications for hypertension. Need to investigate why and what it implies.

Finally, 7 patients in the perfusion datset are taking insulin. Usually patients taking insulin in T2DM are at advanced stage of the disease and therefore more at risk for CVDs.